In [1]:
import yaml
import threading
from threading import Thread
import time

In [2]:
import logging

Log_Format="%(asctime)s.%(msecs)06d;%(message)s"
datefmt='%Y-%m-%d %H:%M:%S'
    
logging.basicConfig(filename = "logfile.log",
                    filemode = "w",
                    format = Log_Format, 
                    datefmt = datefmt,
                    level = logging.INFO)

logger = logging.getLogger()

In [3]:
def convertPath2String(path) :     
    return ".".join(path)

In [4]:
def timeFunction(path, func_name, waiting_time) : 
    
    logger.info( convertPath2String(path) + " Executing TimeFunction(" + func_name + ", " + waiting_time + ")" )
    time.sleep(int(waiting_time))
    logger.info(convertPath2String(path) + " Exit")  

In [5]:
def handleYAML(data, path):

    if(data['Type'] == 'Task') : 

        if(data['Function'] == 'TimeFunction') :
            
            func_name = data['Inputs']['FunctionInput']
            waiting_time = data['Inputs']['ExecutionTime']
            
            timeFunction(path, func_name, waiting_time)

    elif (data['Type'] == 'Flow') :

        orig_path = path
        
        if(data['Execution'] == 'Sequential') :
        
            for activity in data['Activities'] :
                new_key = activity
                
                path.append(new_key)
                logger.info(convertPath2String(path) + " Entry")
                
                new_thread = Thread(target = handleYAML, args=[data['Activities'][new_key], path[:]])
                
                new_thread.start()
                new_thread.join()
                
                path.pop()
                
                
        elif(data['Execution'] == 'Concurrent') :
            
            threads = []
            for activity in data['Activities'] :
                new_key = activity
                path.append(new_key)
                logger.info(convertPath2String(path) + " Entry")
                
                new_thread = Thread(target = handleYAML, args=[data['Activities'][new_key], path[:]])
                new_thread.start()
                
                threads.append((new_thread, path[:]))
                
                path.pop()
                
            for tup in (threads):                
                thread = tup[0]
                thread.join()
            
        logger.info(convertPath2String(orig_path) + " Exit")

In [6]:
with open("Milestone1B.yaml", "r") as stream:
    try:
        data = yaml.safe_load(stream)
        
        path = []
        
        key = list(data.keys())[0]
        
        path.append(key)
        logger.info(convertPath2String(path) + " Entry")
        
        new_thread = Thread(target = handleYAML, args=[data[key], path[:]])
        new_thread.start()
        new_thread.join()
        
                
    except yaml.YAMLError as exc:
        print(exc)